In [1]:
from typing import Union, Self
from numbers import Number
from itertools import chain, product

import numpy as np
import sympy as sp
import cvxpy as cp

import scipy
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from scipy.linalg import qr
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import splu, svds
from sksparse.cholmod import cholesky

from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem, DoubleTraceOperator
from bmn.linear_algebra import get_null_space_dense, create_sparse_matrix_from_dict, get_row_space_dense, get_null_space_sparse
from bmn.bootstrap import BootstrapSystem
from bmn.solver import (
    minimal_eigval,
    sdp_init, sdp_relax,
    sdp_minimize,
    minimize,
    get_quadratic_constraint_vector_sparse,
    get_quadratic_constraint_vector_dense,
    compute_L2_norm_of_quadratic_constraints,
)
from bmn.brezin import compute_Brezin_energy, compute_Brezin_energy_Han_conventions

from bmn.debug_utils import disable_debug

import yaml
from bmn.config_utils import (
    run_bootstrap_from_config,
    generate_configs_one_matrix,
    generate_configs_two_matrix,
    generate_configs_three_matrix,
    generate_configs_bfss,
    )

np.set_printoptions(linewidth=120)  # Adjust the number to the desired width

# plot settings
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5.0
plt.rcParams['xtick.minor.size'] = 3.0
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['lines.linewidth'] = 2
plt.rc('font', family='serif',size=16)
matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=16)
matplotlib.rcParams['axes.prop_cycle'] = cycler(
    color=['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8']
    )
matplotlib.rcParams.update(
    {"axes.grid":True,
    "grid.alpha":0.75,
    "grid.linewidth":0.5}
    )
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

TODO
- consolidate 1, 2, 3, and BFSS models
- consolidate generate_model_X_config 
- move checkpoints, maybe add info on the checkpoints

In [2]:
generate_configs_bfss(
    config_filename=f"test",
    config_dir="MiniBFSS_L_3_test",
    st_operator_to_minimize="x_2",
    st_operators_evs_to_set={"energy": 1},
    load_from_previously_computed=True,
    impose_symmetries=True,
    reg_decay_rate=0.7880462815669912,
    )

# python bmn/config_utils.py run_bootstrap_from_config -config_filename test -config_dir MiniBFSS_L_3_test

In [ ]:
reg_decay_rate = np.exp(-3 * np.log(10) / (30 - 1))
reg_decay_rate**(30-1), reg_decay_rate

In [ ]:
generate_configs_bfss(
    config_filename=f"test",
    config_dir="MiniBFSS_L_3_test",
    st_operators_evs_to_set={"energy": 1},
    load_from_previously_computed=True,
    impose_symmetries=True,
    tol=1e-2,
    maxiters=50,
    maxiters_cvxpy=10_000,
    init_scale=1e2,
    penalty_reg=1e6,
    reg=1e-4,
    #reg_decay_rate=reg_decay_rate,
    )

# python bmn/config_utils.py run_bootstrap_from_config -config_filename test_2 -config_dir MiniBFSS_L_3_test

In [ ]:
1.9525 / 3

In [ ]:
generate_configs_two_matrix(
    config_filename=f"test_small_mass_hold_energy_fixed",
    config_dir="TwoMatrix_L_3_symmetric",
    st_operator_to_minimize="x_2",
    st_operators_evs_to_set={"energy": 1},
    g2=1e-4,
    g4=1,
    load_from_previously_computed=True,
    impose_symmetries=True,
    )

In [ ]:
generate_configs_three_matrix(
    config_filename=f"test_small_mass",
    config_dir="ThreeMatrix_L_3_symmetric",
    g2=1e-4,
    g4=1,
    load_from_previously_computed=False,
    impose_symmetries=True,
    )

In [ ]:
run_bootstrap_from_config(
    config_filename=f"g4_{str(0.448)}",
    config_dir="TwoMatrix_L_4_symmetric",
    verbose=False,
    )

In [ ]:
for reg in [1e3, 1e4, 1e5, 1e6, 1e7, 1e8, 1e9]:

    generate_configs_bfss(
        config_filename=f"reg_{int(reg)}",
        config_dir="MiniBFSS_L_3_symmetric",
        energy=1,
        st_operator_to_minimize="x_2",
        load_from_previously_computed=True,
        impose_symmetries=True,
        maxiters_cvxpy=5_000,
        maxiters=100,
        radius=1e6,
        reg=reg,
        )

In [ ]:
st_operator_to_minimize = "x_2"

generate_configs_bfss(
    "MiniBFSS_symmetric_L_3",
    energy=1,
    st_operator_to_minimize="x_2",
    load_from_previously_computed=True,
    impose_symmetries=True,
    maxiters_cvxpy=10_000,
    maxiters=100,
    radius=1e8,
    )

generate_configs_bfss(
    "MiniBFSS_L_3",
    energy=1,
    st_operator_to_minimize="x_2",
    load_from_previously_computed=True,
    impose_symmetries=False,
    maxiters_cvxpy=10_000,
    maxiters=100,
    radius=1e8,
    )

#run_bootstrap_from_config("hello", verbose=False)

## BFSS

In [ ]:
L = 2

matrix_system = MatrixSystem(
    operator_basis=["X0", "X1", "X2", "Pi0", "Pi1", "Pi2"],
    commutation_rules_concise={
        ("Pi0", "X0"): 1,  # use Pi' = i P to ensure reality
        ("Pi1", "X1"): 1,
        ("Pi2", "X2"): 1,
    },
    hermitian_dict={
        "Pi0": False,
        "X0": True,
        "Pi1": False,
        "X1": True,
        "Pi2": False,
        "X2": True,
        },
)

# lambda = 1 here
hamiltonian = SingleTraceOperator(
    data={
        ("Pi0", "Pi0"): -0.5,
        ("Pi1", "Pi1"): -0.5,
        ("Pi2", "Pi2"): -0.5,
        # quartic term (XY)
        ("X0", "X1", "X0", "X1"): -1 / 4,
        ("X1", "X0", "X1", "X0"): -1 / 4,
        ("X0", "X1", "X1", "X0"): 1 / 4,
        ("X1", "X0", "X0", "X1"): 1 / 4,
        # quartic term (XZ)
        ("X0", "X2", "X0", "X2"): -1 / 4,
        ("X2", "X0", "X2", "X0"): -1 / 4,
        ("X0", "X2", "X2", "X0"): 1 / 4,
        ("X2", "X0", "X0", "X2"): 1 / 4,
        # quartic term (YZ)
        ("X1", "X2", "X1", "X2"): -1 / 4,
        ("X2", "X1", "X2", "X1"): -1 / 4,
        ("X1", "X2", "X2", "X1"): 1 / 4,
        ("X2", "X1", "X1", "X2"): 1 / 4,
    }
)

# <tr G O > = 0
gauge = MatrixOperator(data={
    ("X0", "Pi0"): 1,
    ("Pi0", "X0"): -1,
    ("X1", "Pi1"): 1,
    ("Pi1", "X1"): -1,
    ("X2", "Pi2"): 1,
    ("Pi2", "X2"): -1,
    (): 3,
    })

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge_generator=gauge,
    max_degree_L=L,
    odd_degree_vanish=True,
    simplify_quadratic=True,
)

bootstrap.build_quadratic_constraints()

quadratic_constraints_numerical = bootstrap.quadratic_constraints_numerical

In [ ]:
quadratic_constraints_numerical

In [ ]:
def newton_pseudoinverse(param, nsteps, quadratic_constraints_numerical):

    for step in range(nsteps):
        val, grad = get_quadratic_constraint_vector_sparse(
            quadratic_constraints=quadratic_constraints_numerical,
            param=param,
            compute_grad=True,
            )
        grad_pinv = np.linalg.pinv(grad)
        #np.allclose(grad @ grad_pinv @ grad, grad)
        #np.allclose(grad.T @ np.linalg.inv(grad @ grad.T), grad_pinv)
        param = param - np.asarray(grad_pinv @ val)[0]
        print(f"Newton's method: step = {step}, val = {np.linalg.norm(val)}")

    return param

In [ ]:
bootstrap.param_dim_null

In [ ]:
167958/3

In [ ]:
param = np.random.normal(size=bootstrap.param_dim_null)
val = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    )

param = newton_pseudoinverse(param=param, nsteps=10, quadratic_constraints_numerical=quadratic_constraints_numerical)
val = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    )

In [ ]:
val.shape, grad.T.shape

In [ ]:
val.shape, grad.shape, grad_pinv.shape

In [ ]:
val2, grad2 = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

In [ ]:
np.sum(np.abs(grad2[0]))

In [ ]:
grad2.shape

In [ ]:
min(np.sum(np.abs(np.asarray(grad2)), axis=1)), max(np.sum(np.abs(np.asarray(grad2)), axis=1))

In [ ]:
min(np.abs(val2)), max(np.abs(val2))

In [ ]:
val1, grad1 = get_quadratic_constraint_vector_dense(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

val2, grad2 = get_quadratic_constraint_vector_sparse(
    quadratic_constraints=quadratic_constraints_numerical,
    param=param,
    compute_grad=True,
    )

In [ ]:
np.allclose(val1, val2), np.allclose(grad1, grad2)

## BMN

In [ ]:
from bmn.models import MiniBMN
from bmn.bootstrap_complex import BootstrapSystemComplex

model = MiniBMN(couplings={"g2": 1, "g4": 1})

bootstrap = BootstrapSystemComplex(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=3,
    simplify_quadratic=False,
    symmetry_generators=None,
    fraction_operators_to_retain=0.41,
    )

In [ ]:
bootstrap.build_null_space_matrix()

In [ ]:
bootstrap.build_linear_constraints()

In [ ]:
bootstrap.build_quadratic_constraints()

In [ ]:
bootstrap.quadratic_constraints_numerical["quadratic"][0]

In [ ]:
bootstrap.build_bootstrap_table()

In [ ]:
from bmn.newton_solver import solve_bootstrap

param, optimization_result = solve_bootstrap(
    bootstrap=bootstrap,
    st_operator_to_minimize=bootstrap.hamiltonian,
    )

In [ ]:
x = 1 + 1j
x.real

In [ ]:
x = 1
x.real